# 引言

Python提供了一种特殊的写法，叫做推导(comprehension)，可以简洁地迭代列表、字典和集合等数据结构，并根据迭代结果生成另一套数据。

Python把这种理念也运用到了函数上面，产生了生成器(generator)，它可以让函数每次返回一系列值中的一个。凡是可以使用迭代器的任务都支持生成器函数。

# #27 用列表推导取代map与filter

假设我们要用列表中每个元素的平方值构建一份新的列表，传统的写法是采用`for`循环来写。

In [1]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
squares = []
for x in a:
    squares.append(x**2)
print(squares)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


这段代码可以改用列表推导来写。

In [3]:
squares = [x**2 for x in a]
squares

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

当然也可以用`map`实现该功能，它能从多个列表中分别取出当前位置上的元素，并把它们当作参数传递给映射函数，以求出新列表在这个位置上的元素值。但这看起来有点繁琐。

In [6]:
alt = map(lambda x: x ** 2 , a)

列表推导式还有一个地方比`map`好，就是它能方便地过滤原列表，把某些输入值对应的计算结果从输出结果中排除。

In [8]:
even_squares = [x**2 for x in a if x % 2 == 0]
even_squares

[4, 16, 36, 64, 100]

这种功能也可以通过内置的`filter`与`map`实现，但是这两个函数相结合的写法可读性没那么好。

In [9]:
alt = map(lambda x: x**2, filter(lambda x: x % 2 ==0, a))
assert even_squares == list(alt)

==字典与几何也有相应的推导机制，分别叫字典推导与集合推导。==

In [10]:
even_squares_dict = {x: x**2 for x in a if x % 2 == 0}
threes_cubed_set = {x**3 for x in a if x % 3 == 0}
print(even_squares_dict)
print(threes_cubed_set)

{2: 4, 4: 16, 6: 36, 8: 64, 10: 100}
{216, 729, 27}


如果改用`map`与`filter`实现，那么还必须调用相应的构造器，这会让代码变得很长。

In [11]:
alt_dict = dict(map(lambda x: (x, x**2),
				filter(lambda x: x % 2 == 0, a)))
alt_set = set(map(lambda x: x**3,
	          filter(lambda x: x % 3 == 0, a)))
assert even_squares_dict == alt_dict
assert threes_cubed_set == alt_set

# #28 控制推导逻辑的子表达式不要超过两个

除了最基本的用法外，列表推导式还支持多层循环。例如把矩阵转化成普通的一维列表，那么可以在推导时，使用两条`for`子表达式。这些子表达式会按照从左到右的顺序解读。

In [12]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [x for row in matrix for x in row] #先是得到row，再遍历row得到x
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


多层循环还可以用来重制那种两层深的结构。例如，如果要根据二维矩阵里每个元素的平方值构建一个新的二维矩阵，那么可以采用下面的写法。

In [14]:
squared = [[x**2 for x in row] for row in matrix]
squared

[[1, 4, 9], [16, 25, 36], [49, 64, 81]]

这看上去有点复杂，因为它把小的推导式逻辑`[x**2 for x in row]`嵌到了大的推导逻辑里面。大的推导逻辑用来决定新矩阵里的每一行，小的推导逻辑决定行中的每个元素。

这行语句总体上不难理解。但如果推导过程中还要再加一层循环，那么语句会很长，必须写成多行。比如下面把一个三维矩阵化成普通一维列表。

In [15]:
my_lists = [
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]],
]
flat = [x for sublist1 in my_lists
        for sublist2 in sublist1
        for x in sublist2]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


此时，采用列表推导式来实现，其实并不会比传统的`for`循环节省多少代码。下面用`for`循环来写一次，这要比刚才那种三层矩阵的列表推导式更加清晰。

In [17]:
flat = []
for sublist1 in my_lists:
    for sublist2 in sublist1:
        flat.extend(sublist2)
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


推导的时候，可以使用多个`if`条件。如果这些`if`条件出现在同一层循环内，那么它们之间默认是`and`关系。例如，如果要用原列表中大于4且是偶数的值来构建新列表，那么既可以连用两个`if`，也可以只用一个`if`。

In [18]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = [x for x in a if x > 4 if x % 2 == 0]
c = [x for x in a if x > 4 and x % 2 == 0]
print(b)
print(c)
assert b and c
assert b == c

[6, 8, 10]
[6, 8, 10]


在推导时，每一层的`for`子表达式都可以带有`if`条件。例如，要根据原矩阵构建新的矩阵，把其中各元素之和大于等于10的那些行选出来，而且只保留其中能被3整除的那些元素。
这个逻辑用列表推导来写，并不需要太多的代码，但是会很难理解。

In [19]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
filtered = [[x for x in row if x % 3 ==0]
            for row in matrix if sum(row) >= 10]
filtered

[[6], [9]]

总之，在表示推导逻辑时，最多只应该写两个子表达式(两个`if`条件、两个`for`循环或一个`if`条件和一个`for`循环)。
只要实现的逻辑比这复杂，就应该采用普通的`if`与`for`来实现，且可以考虑编写辅助函数。

# #29 用赋值表达式消除推导中的重复代码

推导列表、字典与集合等变体结构时，经常要在多个地方用到同一个计算结果。例如，我们要给制作紧固件的公司编程以管理订单。顾客下单后，我们要判断当前的库存能否满足这份订单，即，要核查每种产品的数量有没有达到可以发货的最低限制(8个为一批，至少得有一批才能发货)。


In [20]:
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips'] # 库存中无 clip

def get_batches(count, size):
    return count // size

result = {}
for name in order:
    count = stock.get(name, 0)
    batches = get_batches(count, 8)
    if batches:
        result[name] = batches

print(result)

{'screws': 4, 'wingnuts': 1}


这段循环逻辑，如果改用字典推导来写，会简单一些。

In [21]:
found = {name: get_batches(stock.get(name,0), 8)
         for name in order
         if get_batches(stock.get(name,0), 8)}
found

{'screws': 4, 'wingnuts': 1}

这样写虽然简单，但是，`get_batches(stock.get(name,0), 8)`写了两遍。这样会让代码看起来比较乱，而且如果两个地方忘了同步更新，就会出问题。例如，我们决定每一个批次是4个，而不是8个。如果我们忘了同步修改另一个地方，那么代码可能出问题。

In [22]:
has_bug = {name: get_batches(stock.get(name, 0), 4)
           for name in order
           if get_batches(stock.get(name, 0), 8)}

print('Expected:', found)
print('Found:   ', has_bug)

Expected: {'screws': 4, 'wingnuts': 1}
Found:    {'screws': 8, 'wingnuts': 2}


有个简单的方法可以解决这个问题，那就是在推导的过程中使用Python3.8引入的`:=`海象操作符进行赋值表达。

In [25]:
found = {name: batches for name in order if (batches := get_batches(stock.get(name, 0),8))}
found

{'screws': 4, 'wingnuts': 1}

`batches := get_batches(...)`这条赋值表达式，能从`stock`字典里查找到对应产品一共有几批，并把这个批数放到`batches`变量里。这样，我们推导这个产品所对应批数时，就不用再通过`get_batches`计算了。

在推导过程中，描述新值的那一部分也可以出现赋值表达式。但如果在其他部分引用了定义在那一部分的变量，那么程序可能就会在运行时出错。
例如，如果写成了下面这样，那么程序就必须先考虑`for name, count in stock.items() if tenth > 0`，而这个时候，其中`tenth`还未定义。

In [27]:
result = {name: (tenth := count // 10)
          for name, count in stock.items() if tenth > 0}

NameError: name 'tenth' is not defined

为了解决这个问题，可以把赋值表达式移动到`if`条件里，然后描述新值的这一部分引用已经定义过的`tenth`变量。

In [28]:
result = {name: tenth for name, count in stock.items()
          if (tenth := count // 10) > 0}
result

{'nails': 12, 'screws': 3, 'washers': 2}

如果推导逻辑不带条件，而表示新值的那一部分又使用了`:=`操作符，那么操作符左边的变量就会泄漏到包含这条推导语句的那个作用域里。

In [29]:
half = [(last := count // 2) for count in stock.values()]
print(f'Last item of {half} is {last}')

Last item of [62, 17, 4, 12] is 12


这与普通的`for`循环所用的那个循环变量类似。

In [30]:
for count in stock.values():  # Leaks loop variable
    pass
print(f'Last item of {list(stock.values())} is {count}')

Last item of [125, 35, 8, 24] is 24


然而，推导语句中的`for`循环所使用的循环变量，是不会像刚才那样泄漏到外面的。

In [31]:
del count
half = [count // 2 for count in stock.values()]
print(half)   # Works
print(count)  # Exception because loop variable didn't leak

[62, 17, 4, 12]


NameError: name 'count' is not defined

最好不要泄漏循环变量，所以，建议赋值表达式只出现在推导逻辑的条件之中。

赋值表达式不仅可以用在推导过程中，而且可以用来编写生成器表达式。下面这种写法创建的是迭代器，而不是字典实例，该迭代器每次会给出一对数值，其中第一个元素为产品的名字，第二个元素为这种产品的库存。

In [33]:
found = ((name, batches) for name in order
         if (batches := get_batches(stock.get(name, 0), 8)))
print(next(found))
print(next(found))
print(batches) # 还是会跑到生成器表达式外部，注意！！

('screws', 4)
('wingnuts', 1)
1


# #30 不要让函数直接返回列表参数，应该让它逐个生成列表里的值

如果函数要返回的是包含许多结果的序列，那么最简单的办法就是把这些结果放到列表中。例如，我们要返回字符串里每个单词的首字母所对应的下标。

In [34]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

我们把一段文本传给这个函数，可以返回正确的结果。

In [35]:
address = 'Four score and seven years ago...'
result = index_words(address)
result[:10]

[0, 5, 11, 15, 21, 27]

但是该函数有两个缺点。

第一个缺点是，它的代码看起来有点乱。这种函数改用生成器来实现会比较好。生成器由包含`yield`表达式的函数创建。下面就定义一个生成器函数，实现与刚才那个函数相同的效果。

In [37]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

调用生成器函数并不会让其中的代码立刻得到执行，它会返回一个迭代器。把迭代器传给内置的`next`函数，就可以将生成器函数推进到它的下一条`yield`表达式。
生成器会把`yield`表达式的值通过迭代器返回给调用者。

In [38]:
it = index_words_iter(address)
print(next(it))
print(next(it))

0
5


这次的`index_words_iter`函数，比刚才那个函数好懂很多。
如果确实要制作一份列表返回，那可以把生成器函数返回的迭代器传给内置的`list`函数。

In [39]:
result = list(index_words_iter(address))
result[:10]

[0, 5, 11, 15, 21, 27]

`index_words`的第二个缺点是，它必须把所有的结果都保存到列表中。然后才能返回列表。如果输入的数据特别多，那么程序可能会因为内存不足而崩溃。

相反，用生成器函数来实现，就不会有这个问题了。它可以接受长度任意的输入信息，并把内存消耗量压得较低。例如下面这个生成器，只需要把当前这行文字从文件中读进来就行，每次推进的时候，它都只处理一个单词，直到把当前这行文字处理完毕，才读入下一行文字。

In [40]:
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

该函数运行时所耗的内存，取决于文件中最长的那一行所包含的字符数。

定义这种生成器函数的时候，有一个要注意的地方，就是调用者无法重复使用函数所返回的迭代器，因为这些迭代器是有状态的。

# #31 谨慎地迭代函数所收到的参数

如果函数接受的参数是个包含许多对象的列表，那么这份列表有可能要迭代多次。例如，我们要分析美国得克萨斯州的游客数量。原始数据保存在一份列表中，其中每个元素表示每年有多少游客到这个城市旅游。我们现在要统计每个城市的游客数占游客总数的百分比。

为了求出这份数据，我们编写一个归一化函数`normalzie`，它先把列表里的所有元素加起来求出游客总数，然后，分别用每个城市的游客数除以游客总数计算出该城市在总数中所占的百分比。

In [41]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

把一份范例数据放到列表中传给这个函数，可以得到正确的结果。

In [42]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


为了应对规模更大的数据，我们现在需要让程序能从文件中读取信息，并假设数据都在这份文件中。我们通过生成器实现。

In [45]:
path = 'my_numbers.txt'
with open(path, 'w') as f:
    for i in (15, 35, 80):
        f.write('%d\n' % i)

def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

奇怪的是，对`read_visits`所返回的迭代器调用`normalize`函数后，并没有得到结果。

In [46]:
it = read_visits(path)
percentages = normalize(it)
print(percentages)

[]


出现这种情况的原因在于，迭代器只能产生一个结果。假如迭代器或生成器已经跑出过`StopIteration`异常，继续用它来构造列表或是像`normalize`那样对它做`for`循环，那它不会给出任何结果。

In [47]:
it = read_visits(path)
print(list(it))
print(list(it)) # 已经被使用了

[15, 35, 80]
[]


还有个因素也很令人迷惑，那就是：在已经把数据耗完的迭代器上继续迭代，程序居然不报错。

为了解决这个问题，我们可以把外界传入的迭代器特意遍历一整轮，从而将其中的内容复制到一份列表中。这样的话，就可以用这份列表来处理数据了。

In [48]:
def normalize_copy(numbers):
    numbers_copy = list(numbers)  # Copy the iterator
    total = sum(numbers_copy)
    result = []
    for value in numbers_copy:
        percent = 100 * value / total
        result.append(percent)
    return result

现在的这个函数就可以正确处理`read_visits`生成器函数所返回的`it`值了。

In [49]:
it = read_visits('my_numbers.txt')
percentages = normalize_copy(it)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


这个办法虽然能解决问题，但如果输入的迭代器所要提供的数据量很大。那么就可能导致内存耗尽。
为了应对大规模的数据，其中一个变通方案是让`normalize`函数接受另外一个函数，使它每次要使用迭代器时，都去向那个函数索要。

In [50]:
def normalize_func(get_iter):
    total = sum(get_iter())   # New iterator
    result = []
    for value in get_iter():  # New iterator
        percent = 100 * value / total
        result.append(percent)
    return result

使用`normalize_func`时，需要传入一条`lambda`表达式，让这个表达式去调用`read_visits`生成器函数。这样`normalize_func`每次向`get_iter`索要迭代器时，程序都会给出一个新的迭代器。

In [51]:
percentages = normalize_func(lambda: read_visits(path))
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


这样虽然可行，但是传入这么一个`lambda`表达式显得有点生硬。要想用更好的办法解决这个问题，可以新建一种容器类，让它实现迭代器协议。

Python的`for`循环及相关的表达式，正是按照迭代器协议来遍历容器内容的。Python执行`for x in foo`这样的语句时，实际上会调用`iter(foo)`。这个函数会触发`foo.__iter__`的特殊方法，该方法必须返回迭代器对象。最后，Python会用迭代器对象反复调用内置的`next`函数，直到数据耗尽位置(如果抛出`StopIteration`异常，就表示数据已经迭代完了)。

总结起来，只需要让你的类在实现`__iter__`方法时，按照生成器的方式来写就好。

In [58]:
class ReadVisits:
    def __init__(self, data_path):
        self.data_path = data_path
    
    def __iter__(self):
        print('Call __iter__')
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

我们只需要把新的容器传给最早的那个`normalzie`函数运行即可，函数本身的代码不需要修改。

In [53]:
visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


这样做为什么可以呢？因为`normalize`的`sun`会触发`ReadVisits.__iter__`让系统分配一个新的迭代器对象给它。接下来，`normalize`通过`for`循环计算每项数据占总值的百分比时，又会触发`__iter__`，于是系统又分配另一个迭代器对象。这两个迭代器对象互不影响。这种方案唯一的缺点，就是多次读取输入数据。

明白了`ReadVisits`这种容器的工作原理后，我们就可以在编写函数和方法时先确认一下，收到的应该是像`ReadVisits`这样的容器，而不是普通的迭代器。
如果将普通的迭代器传给内置的`iter`函数，那么函数就会把迭代器本身返回给调用者。反之，如果传来的是容器类型，那么`iter`函数就会返回一个新的迭代器对象。
我们可以借助这条规律，判断输入值是不是这种容器。如果不是，就抛出`TpyeError`异常，因为我们没办法在那样的值上重复遍历。

In [54]:
def normalize_defensive(numbers):
    if iter(numbers) is numbers:  # 说明是迭代器对象
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

还有一种写法也可以检测出这样的问题。`collections.abc`内置模块里定义了名为`Iterator`的类，它用在`isinstance`函数中，可以判断自己收到的参数是不是这种实例。

In [55]:
from collections.abc import Iterator 

def normalize_defensive(numbers):
    if isinstance(numbers, Iterator):  # 如果是，抛出异常
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

这种自定义容器的方案，最适合于既不想像`normalize_copy`函数一样，把迭代器提供的这套数据全部复制一份，同时又要多次遍历这套数据的情况。
该方案不仅支持自定义容器，还支持系统自带的列表类型，因为这些全都属于遵循迭代器协议的可迭代容器。

In [59]:
visits = [15, 35, 80]
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

visits = ReadVisits(path)
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

Call __iter__
Call __iter__


如果输入的是普通迭代器而不是容器，那么`normalize_defensive`就会抛出异常。

In [57]:
visits = [15, 35, 80]
it = iter(visits)
normalize_defensive(it)

TypeError: Must supply a container